In [1]:
import sys
sys.path.append("..")

# backends
from qiskit.providers.fake_provider import Fake27QPulseV1,Fake5QV1,Fake20QV1
from qiskit_ibm_runtime.fake_provider import FakeGuadalupeV2


# poly_sabre
from poly_sabre.poly_circuit_preprocess import *
from poly_sabre.poly_circuit_utils import *
from poly_sabre.poly_sabre import POLY_SABRE


# qiskit
from qiskit import QuantumCircuit, transpile
from qiskit.transpiler.passes import SabreLayout, SabreSwap
from qiskit.transpiler import PassManager
from qiskit.transpiler import CouplingMap



In [2]:
file_path = "../benchmarks/queko-bss-16qbt/16QBT_100CYC_QSE_0.json"
data = json_file_to_isl(file_path)
edges = FakeGuadalupeV2().configuration().coupling_map

In [3]:
poly_sabre = POLY_SABRE(edges,data)

In [4]:
initial_mapping = poly_sabre.get_initial_mapping()

In [5]:
poly_sabre.run(initial_mapping,num_iter=1)

Executing Gates: 10it [00:00, 712.42it/s]             
Executing Gates: 100%|██████████| 20/20 [00:00<00:00, 144.81it/s]
Executing Gates: 13it [00:00, 107.30it/s]                        
Executing Gates: 12it [00:00, 111.22it/s]              
Executing Gates: 100%|██████████| 12/12 [00:00<00:00, 117.17it/s]
Executing Gates: 13it [00:00, 157.71it/s]              
Executing Gates: 14it [00:00, 147.69it/s]              
Executing Gates: 15it [00:00, 90.76it/s]                       
Executing Gates: 16it [00:00, 72.34it/s]                        
Executing Gates: 10it [00:00, 473.51it/s]             
Executing Gates: 100%|██████████| 18/18 [00:00<00:00, 101.58it/s]
Executing Gates: 11it [00:00, 110.59it/s]             
Executing Gates: 13it [00:00, 89.23it/s]                       
Executing Gates: 11it [00:00, 213.50it/s]             
Executing Gates: 100%|██████████| 12/12 [00:00<00:00, 108.63it/s]
Executing Gates: 12it [00:00, 100.39it/s]                       
Executing Gates: 100%|██

**************************************************
total swaps in iteration 0 is  291
**************************************************


291

In [10]:
circuit = QuantumCircuit.from_qasm_str(data["qasm_code"])
coupling_map = CouplingMap(edges)

sabre_layout = SabreLayout(coupling_map,seed=21)
pm_initial_layout = PassManager(sabre_layout)
mapped_circuit = pm_initial_layout.run(circuit)

sabre_swap = SabreSwap(coupling_map,seed=21)
pm_swap = PassManager(sabre_swap)
routed_circuit = pm_swap.run(mapped_circuit)

swap_count = routed_circuit.count_ops().get('swap', 0)
print(swap_count)

158


In [ ]:
transpiled_circuit = transpile(circuit, coupling_map=edges,seed_transpiler=21, optimization_level=0)
sabre_swap_count = transpiled_circuit.count_ops().get('swap',0) 
print(sabre_swap_count)